## Configuration and Setup


In [1]:
import os
os.environ['HF_HOME'] = "C:/HF_CACHE/"

In [2]:
import torch

from cs324_project.datasets import GlueDatasetTask, load_glue_dataset_info
from cs324_project.models import ModelCheckpointName, load_classification_model, load_pretraining_model, load_tokenizer
from cs324_project.masking import (
    get_training_args_mlm, get_trainer_mlm, RandomMaskingConfig, WholeWordMaskingConfig, TyphoonMaskingConfig)
from cs324_project.classification import get_training_args_sc, get_trainer_sc

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
model_name = ModelCheckpointName.TINYBERT_HUAWEI
task = GlueDatasetTask.MRPC

In [5]:
tokenizer = load_tokenizer(model_name)
dataset_info = load_glue_dataset_info(task, tokenizer)

Found cached dataset glue (C:/HF_CACHE/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


  0%|          | 0/3 [00:00<?, ?it/s]

Loading cached processed dataset at C:\HF_CACHE\datasets\glue\mrpc\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad\cache-210d87aa331c9a9e.arrow
Loading cached processed dataset at C:\HF_CACHE\datasets\glue\mrpc\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad\cache-33abd690c2474663.arrow
Loading cached processed dataset at C:\HF_CACHE\datasets\glue\mrpc\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad\cache-fd99a719fa52af0a.arrow
Loading cached processed dataset at C:\HF_CACHE\datasets\glue\mrpc\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad\cache-98cf444ee3350795.arrow
Loading cached processed dataset at C:\HF_CACHE\datasets\glue\mrpc\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad\cache-9383850b1455b190.arrow
Loading cached processed dataset at C:\HF_CACHE\datasets\glue\mrpc\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad\cache-bcdcf9ec1f3ace0e.arro

## Fine-tune with masking

In [6]:
model_mlm = load_pretraining_model(model_name, dataset_info)

Some weights of the model checkpoint at huawei-noah/TinyBERT_General_4L_312D were not used when initializing BertForMaskedLM: ['fit_denses.2.bias', 'fit_denses.2.weight', 'fit_denses.0.bias', 'cls.seq_relationship.bias', 'fit_denses.3.weight', 'fit_denses.1.weight', 'cls.seq_relationship.weight', 'fit_denses.1.bias', 'fit_denses.3.bias', 'fit_denses.4.weight', 'fit_denses.0.weight', 'fit_denses.4.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [7]:
training_args_mlm = get_training_args_mlm(
    masking_config=TyphoonMaskingConfig(),
    num_epochs=100)
trainer_mlm = get_trainer_mlm(
    dataset_info=dataset_info,
    mlm_args=training_args_mlm,
    model=model_mlm)

Creating training arguments, model output dir: C:\Users\Windows\Desktop\Shahir\cs324-final-project-2023\models\mlm\Model 03-22-2023 05-57-13 AM


In [8]:
trainer_mlm.train()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,6.431400,5.714137
2,5.210800,4.945677
3,4.706200,4.574192
4,4.363100,4.323779
5,4.137800,4.185390
6,3.953700,3.983167
7,3.794200,3.920470
8,3.638600,3.790572
9,3.578800,3.812777
10,3.444600,3.732371


TrainOutput(global_step=23000, training_loss=2.2991143931513247, metrics={'train_runtime': 2193.9784, 'train_samples_per_second': 167.185, 'train_steps_per_second': 10.483, 'total_flos': 786121535232432.0, 'train_loss': 2.2991143931513247, 'epoch': 100.0})

In [9]:
trainer_mlm.evaluate()

{'eval_loss': 3.381281852722168,
 'eval_runtime': 0.5495,
 'eval_samples_per_second': 742.453,
 'eval_steps_per_second': 47.313,
 'epoch': 100.0}

In [10]:
print(trainer_mlm.state.best_model_checkpoint)

C:\Users\Windows\Desktop\Shahir\cs324-final-project-2023\models\mlm\Model 03-22-2023 05-57-13 AM\checkpoint-11270


## Fine-tune on sequence classification task

In [11]:
model_sc = load_classification_model(trainer_mlm.state.best_model_checkpoint, dataset_info)

Some weights of the model checkpoint at C:\Users\Windows\Desktop\Shahir\cs324-final-project-2023\models\mlm\Model 03-22-2023 05-57-13 AM\checkpoint-11270 were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequ

In [12]:
training_args_sc = get_training_args_sc(
    task,
    learning_rate=1e-5,
    num_epochs=100)
trainer_sc = get_trainer_sc(
    dataset_info=dataset_info,
    model=model_sc,
    training_args=training_args_sc)

Creating training arguments, model output dir: C:\Users\Windows\Desktop\Shahir\cs324-final-project-2023\models\sc\Model 03-22-2023 06-33-50 AM


In [13]:
trainer_sc.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.602600,0.554373,0.710784,0.822823
2,0.519100,0.499403,0.774510,0.851613
3,0.456300,0.476349,0.786765,0.849741
4,0.403800,0.481644,0.750000,0.811111
5,0.365500,0.506840,0.742647,0.797688
6,0.330700,0.535124,0.737745,0.789784
7,0.293500,0.647481,0.686275,0.725322
8,0.267600,0.595161,0.735294,0.793893
9,0.238500,0.693600,0.708333,0.757637
10,0.207700,0.838051,0.686275,0.733333


TrainOutput(global_step=23000, training_loss=0.06432346665114165, metrics={'train_runtime': 743.6428, 'train_samples_per_second': 493.248, 'train_steps_per_second': 30.929, 'total_flos': 781014765977712.0, 'train_loss': 0.06432346665114165, 'epoch': 100.0})

In [14]:
trainer_sc.evaluate()

{'eval_loss': 0.47634875774383545,
 'eval_accuracy': 0.7867647058823529,
 'eval_f1': 0.849740932642487,
 'eval_runtime': 0.2613,
 'eval_samples_per_second': 1561.422,
 'eval_steps_per_second': 99.502,
 'epoch': 100.0}

In [15]:
print(trainer_sc.state.best_model_checkpoint)

C:\Users\Windows\Desktop\Shahir\cs324-final-project-2023\models\sc\Model 03-22-2023 06-33-50 AM\checkpoint-690
